In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
import fnmatch
import os

experiment = 'mom6_solo_global_ALE_z/'


def listdir(dirname, pattern="*"):
    return fnmatch.filter(os.listdir(dirname), pattern)

def findfiles(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result


stdouts = findfiles("stdout*",experiment)

print(stdouts)

['mom6_solo_global_ALE_z/stdout.ubuntu.Inspiron15i7.pgi18.prod.n4', 'mom6_solo_global_ALE_z/stdout.gaea4-intel18.prod.n4', 'mom6_solo_global_ALE_z/stdout.tiger-intel18.prod.n4', 'mom6_solo_global_ALE_z/stdout.googcp-intel19.prod.n6', 'mom6_solo_global_ALE_z/stdout.googcp-intel19.repro.n8', 'mom6_solo_global_ALE_z/stdout.tiger-intel18.prod.n16', 'mom6_solo_global_ALE_z/stdout.gaea4-intel18.prod.n16', 'mom6_solo_global_ALE_z/stdout.ubuntu.Inspiron15i7.gnu7.prod.n4', 'mom6_solo_global_ALE_z/stdout.ubuntu.Inspiron15i7.gnu7.repro.n4', 'mom6_solo_global_ALE_z/stdout.tiger-intel18.prod.n8', 'mom6_solo_global_ALE_z/stdout.gaea4-intel18.prod.n64', 'mom6_solo_global_ALE_z/stdout.ubuntu.Inspiron15i7.gnu7.prod.n6', 'mom6_solo_global_ALE_z/stdout.ubuntu.xpsCentrioDuo.gnu7.repro.n2', 'mom6_solo_global_ALE_z/stdout.ubuntu.Inspiron15i7.gnu7.prod.n8', 'mom6_solo_global_ALE_z/stdout.googcp-intel19.prod.n16', 'mom6_solo_global_ALE_z/stdout.googcp-intel19.prod.n8', 'mom6_solo_global_ALE_z/stdout.tiger-int

In [59]:
import re
from collections import defaultdict

key = 'Main loop'
timing_regex ='\s*' + re.escape(key) + r'\s* ([-+]?\d*\.\d+|\d+)\s*([-+]?\d*\.\d+|\d+)\s*([-+]?\d*\.\d+|\d+)\s*([-+]?\d*\.\d+|\d+)\s*([-+]?\d*\.\d+|\d+)\s*(\d+)\s*(\d+)\s*(\d+)'

table=defaultdict(list)
for stdout in stdouts:
    n = re.match('(\S+)\/stdout.(.*).n\d+',stdout)
    exp = n.group(1)
    plat= n.group(2)
    for line in open(stdout,"r"):
        m = re.match(timing_regex,line)
        if m:
#            print(1+int(m.group(8)),float(m.group(2)))
            table[plat].append((1+int(m.group(8)),float(m.group(2))))


print(table)           

defaultdict(<class 'list'>, {'ubuntu.Inspiron15i7.pgi18.prod': [(4, 110.323791), (6, 119.256983), (8, 115.55814)], 'gaea4-intel18.prod': [(4, 59.304927), (16, 23.654884), (64, 7.836283), (32, 13.162969), (6, 44.48349), (8, 39.907557)], 'tiger-intel18.prod': [(4, 53.044394), (16, 18.535831), (8, 33.245234), (32, 12.647945)], 'googcp-intel19.prod': [(6, 56.768975), (16, 29.161566), (8, 48.68991), (4, 84.808978)], 'googcp-intel19.repro': [(8, 49.197386)], 'ubuntu.Inspiron15i7.gnu7.prod': [(4, 104.994538), (6, 113.068782), (8, 107.94501)], 'ubuntu.Inspiron15i7.gnu7.repro': [(4, 144.16903)], 'ubuntu.xpsCentrioDuo.gnu7.repro': [(2, 4086.224987)], 'ubuntu.xpsCentrioDuo.gnu7.prod': [(2, 3901.233288)], 'ubuntu.Inspiron15i7.pgi18.repro': [(4, 116.327085)]})
